In [8]:
!pip install sagemaker

In [90]:
import sagemaker
sess = sagemaker.Session()
role = sagemaker.get_execution_role()


print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")
#Add instructions for local environment later, if needed

sagemaker role arn: arn:aws:iam::570106654206:role/Dev
sagemaker bucket: sagemaker-us-west-2-570106654206
sagemaker session region: us-west-2


In [65]:
!pygmentize code/train_ptddp_mnist.py

# Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file
# except in compliance with the License. A copy of the License is located at
#
# http://aws.amazon.com/apache2.0/
#
# or in the "LICENSE.txt" file accompanying this file. This file is distributed on an "AS IS"
# BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, express or implied. See the License for
# the specific language governing permissions and limitations under the License.
from __future__ import print_function
import os
import argparse
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import smdistributed.dataparallel.torch.torch_smddp
from torch.distribute

In [91]:
dlc_account_id = 763104351884  # By default, set the account ID used for most regions
region = "us-west-2"
image = (
    "ptddp_image"  # Example: pt-smdataparallel-efficientnet-sagemaker
)
tag = "latest"  # Example: latest

In [59]:
!pygmentize ./Dockerfile

ARG region

# Download base PT DLC. Note that this notebook requires a SM DLC with >= PT 1.10.2
FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.11.0-gpu-py38-cu113-ubuntu20.04-sagemaker

ARG WORK_DIR="ptddp_build"
WORKDIR $WORK_DIR
RUN pwd; pip install git+https://github.com/vishwakaria/sagemaker-pytorch-training-toolkit/; echo "installed pt toolkit"; \
    cd sagemaker-pytorch-training-toolkit; \
    python setup.py; \
    cd ..; pip install git+https://github.com/vishwakaria/sagemaker-python-sdk/; \
    cd sagemaker-python-sdk; \
    python setup.py; \
    cd ../..; rm -rf $WORK_DIR;


In [60]:
!pygmentize ./build_and_push.sh

#!/usr/bin/env bash
# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.
# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# set region

DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
echo "Dir: ${DIR}"
#cd ${DIR}/deberta

if [ "$#" -eq 3 ]; then
    region=$1
    image=$2
    tag=$3
else
    echo "usage: $0 <aws-region> $1 <image-repo> $2 <image-tag>"
    exit 1
fi
# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)
if [ $? -ne 0 ]
then
    exit 255
fi

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"
# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    echo "creating ECR

In [ ]:
! docker system prune -af

In [102]:
region = "us-west-2"
# image = (
#     "ptddp_image"  # Example: pt-smdataparallel-efficientnet-sagemaker
# )
! aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 570106654206.dkr.ecr.{region}.amazonaws.com
#! chmod +x build_and_push.sh
#! bash build_and_push.sh {region} {image} {tag}

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [111]:
from sagemaker.pytorch import PyTorch

# refer https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-training-containers to get the right uri's based on region
#image_uri = '763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.9.1-transformers4.12.3-gpu-py38-cu111-ubuntu20.04'
image_uri = '570106654206.dkr.ecr.us-west-2.amazonaws.com/ptddp-launcher:latest'

# configuration for running training on smdistributed Data Parallel
# this is the only line of code change required to leverage SageMaker Distributed Data Parallel
distribution = {'pytorchddp':{ 'enabled': True }}
#distribution = {"mpi":{"enabled":True, "num_of_processes_per_host":8}}


estimator = PyTorch(
    base_job_name="ptddp-mnist-test",
    source_dir="code",
    entry_point="train_ptddp_mnist.py",
    role=role,
    framework_version="1.11.0",
    py_version="py38",
    image_uri=image_uri,
    # For training with multinode distributed training, set this count. Example: 2
    instance_count=1,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type="ml.p4d.24xlarge",
    sagemaker_session=sess,
    # Training using SMDataParallel Distributed Training Framework
    distribution=distribution,
    debugger_hook_config=False,
)





In [113]:
estimator.fit()

2022-06-28 05:41:08 Starting - Starting the training job...ProfilerReport-1656394868: InProgress
...
2022-06-28 05:41:51 Starting - Preparing the instances for training..........................................
2022-06-28 05:49:10 Downloading - Downloading input data
2022-06-28 05:49:10 Training - Downloading the training image...........................
2022-06-28 05:53:36 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
sed: can't read changehostname.c: No such file or directory
gcc: error: changehostname.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
gcc: error: changehostname.o: No such file or directory
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
/opt/conda/lib/python3.8/site-packages/paramiko/transport.py:236: CryptographyDeprecationW

UnexpectedStatusException: Error for Training job ptddp-mnist-test-2022-06-28-05-41-08-016: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "raise KeyError(key) from None
 KeyError: 'WORLD_SIZE'"
Command "/opt/conda/bin/python3.8 train_ptddp_mnist.py", exit code: 1

In [117]:
region = "us-west-2"
! aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 570106654206.dkr.ecr.{region}.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [120]:
## Test if normal docker container works with this script
# image: PT training DLC (without any code changes)
# distribution = smdistributed, backend = nccl

from sagemaker.pytorch import PyTorch

# refer https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-training-containers to get the right uri's based on region
image_uri = '763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.11.0-gpu-py38-cu113-ubuntu20.04-sagemaker'

# configuration for running training on smdistributed Data Parallel
# this is the only line of code change required to leverage SageMaker Distributed Data Parallel
distribution = { "smdistributed": { "dataparallel": { "enabled": True } } }
#distribution = {"mpi":{"enabled":True, "num_of_processes_per_host":8}}


estimator_pt_base = PyTorch(
    base_job_name="ptddp-mnist-test",
    source_dir="code",
    entry_point="train_ptddp_mnist.py",
    role=role,
    framework_version="1.11.0",
    py_version="py38",
    image_uri=image_uri,
    # For training with multinode distributed training, set this count. Example: 2
    instance_count=1,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type="ml.p4d.24xlarge",
    sagemaker_session=sess,
    # Training using SMDataParallel Distributed Training Framework
    distribution=distribution,
    debugger_hook_config=False,
)


In [122]:
estimator_pt_base.fit()

2022-06-29 00:18:47 Starting - Starting the training job...ProfilerReport-1656461927: InProgress
...
2022-06-29 00:19:35 Starting - Preparing the instances for training.............................................
2022-06-29 00:27:13 Downloading - Downloading input data
2022-06-29 00:27:13 Training - Downloading the training image.....................
2022-06-29 00:30:51 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
INFO:sagemaker-training-toolkit:No exception classes found in smdistributed.dataparallel
INFO:sagemaker-training-toolkit:Imported framework sagemaker_pytorch_container.training
INFO:sagemaker_pytorch_container.training:Block until all host DNS lookups succeed.
INFO:sagemaker_pytorch_container.training:Invoking SMDataParallel
INFO:sagemaker_pytorch_container.training:Invoking user training script.
INFO:sagemaker-training-toolkit:Starting MPI r